In [4]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
import sympy

In [5]:
def generate_lwe_data_batch(n, q, num_samples=32):
    s = np.random.randint(2, size=n)
    a = np.random.randint(0, q, size=(num_samples, n))
    b = (a.dot(s) + np.random.randint(0, 2, size=num_samples)) % q
    return a, b

def build_generator(n, latent_dim, q):
    model = models.Sequential()
    model.add(layers.InputLayer(input_shape=(latent_dim,)))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(32, activation='relu'))
    model.add(layers.Dense(n, activation='linear'))
    return model

def build_discriminator(n):
    model = models.Sequential()
    model.add(layers.InputLayer(input_shape=(n,)))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(32, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))
    return model

def build_gan(generator, discriminator):
    discriminator.trainable = False
    model = models.Sequential()
    model.add(generator)
    model.add(discriminator)
    return model

def compile_models(generator, discriminator, gan):
    discriminator.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    gan.compile(optimizer='adam', loss='binary_crossentropy')

def train_gan(generator, discriminator, gan, n, q, latent_dim=10, epochs=10000, batch_size=32):
    compile_models(generator, discriminator, gan)

    for epoch in range(epochs):
        real_data = generate_lwe_data_batch(n, q, batch_size)
        fake_noise = np.random.rand(batch_size, latent_dim)
        fake_data = generator.predict(fake_noise)
        real_labels = np.ones((batch_size, 1))
        fake_labels = np.zeros((batch_size, 1))

        d_loss_real = discriminator.train_on_batch(real_data[0], real_labels)
        d_loss_fake = discriminator.train_on_batch(fake_data, fake_labels)

        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        valid_noise = np.random.rand(batch_size, latent_dim)
        valid_labels = np.ones((batch_size, 1))

        g_loss = gan.train_on_batch(valid_noise, valid_labels)

        if epoch % 100 == 0:
            print(f"Epoch {epoch}, D Loss: {d_loss[0]}, G Loss: {g_loss}")

In [ ]:
np.random.seed(42)
tf.random.set_seed(42)

# Define the LWE parameters
n = 32
q = sympy.randprime(2**15, 2**30)  # Replace this with the desired prime number
latent_dim = 32  # Adjust the latent dimension as needed

# Build and compile the generator, discriminator, and GAN
generator = build_generator(n, latent_dim, q)
discriminator = build_discriminator(n)
gan = build_gan(generator, discriminator)

# Train the GAN
train_gan(generator, discriminator, gan, n, q, latent_dim=latent_dim, epochs=10000, batch_size=32)

In [ ]:
# Generate fake LWE samples using the trained generator
num_fake_samples = 10
fake_samples = generator.predict(np.random.rand(num_fake_samples, latent_dim))

# Test the discriminator on real and fake samples
real_samples = generate_lwe_data_batch(n, q, num_samples=num_fake_samples)
real_accuracy = discriminator.evaluate(real_samples[0], np.ones((num_fake_samples, 1)), verbose=0)[1]
fake_accuracy = discriminator.evaluate(fake_samples, np.zeros((num_fake_samples, 1)), verbose=0)[1]

print(f"Discriminator Accuracy on Real Samples: {real_accuracy}")
print(f"Discriminator Accuracy on Fake Samples: {fake_accuracy}")